Tensor adalah struktur data data umum berbasis array berdimensi N yang menyimpan dan memanipulasi data matematis.

In [ ]:
# Tensor's class definition code 
import numpy as np

class Tensor(np.ndarray):
    def __new__(cls, input_array) -> Tensor:
        return np.asarray(input_array, dtype=tensor_type()).view(cls)
    # tensor_type() memungkinkan mengganti tipe data kompleks( np.complex64 -> np.complex128)
    # untuk menyesuaikan antara kecepatan, penggunaan memori, dan tingkat akurasi yang diinginkan circuit
    
    def __array_finalize__(self, obj) -> None:
        if obj is None : return


| **Tipe Data** | **Total Bit** | **Komponen** | **Presisi** | **Kegunaan Utama** |
|-----------|----------------|------------------|-----------------|----------------|
| `complex64` | 64 bit (8 byte) | float32 + float32 | ~7 digit desimal | Cepat, hemat memori |
| `complex128` | 128 bit (16 byte) | float64 + float64 | ~15–16 digit desimal | Akurat, tapi berat |
| `complex256` *(opsional di CPU tertentu)* | 256 bit (32 byte) | float128 + float128 | ~33 digit desimal | Super presisi, tapi lambat |
| `bfloat16` *(bisa jadi bagian dari kompleks juga)* | 16 bit (2 byte) | bfloat16 + bfloat16 | ~2–3 digit desimal | Sangat cepat, tapi kasar (approximate) |

untuk tipe _*bfloat*_ menggunakan format floating point: 

$\text{nilai} = (-1)^{\text{sign}} \times (1.\text{mantissa}) \times 2^{(\text{exponent} - \text{bias})}$

sign bit : menyimpan tanda<br>
exponent bits :  menentukan skalar<br> 
mantissa (fraction) bits : menyimpan angka signifikan => menentukan ketelitian<br>


In [ ]:
# How to control type data structure

# bit width of complex data type, 64 or 128. <- will be variable global (saklar for data type)
tensor_width = 64

# All math in this package will use this base type
# Valid values can be np.complex128 or np.complex64
def tensor_type ():
    """Return complex type"""

    if tensor_width == 64:
        return np.complex64
    return np.complex128

### Knocker Product (tensor product)
sebuah operasi pada dua matriks yang menghasilkan matriks naru yang lebih besar (Matriks blok.) <br>

jika matriks A berukuran m x n dan matriks B berukuran p x q <br>
$A \otimes B$ akan berukuran (m x p) x (n x q)

In [ ]:
# how to make a kron function

def kron(self, arg: Tensor) -> Tensor :
    """Return the Kronecker product of this object with arg"""

    return self.__class__(np.kron(self, arg))
def __mul__(self, arg:Tensor) -> Tensor :
    """Inline * operator maps"""

    return self.kron(arg)

Quantum computing sering membuat matriks ukuran besar dengan menggabungkan banyak matriks identik melalui operasi tensor (memanggil fungsi kron berulang) <br>
$U \otimes U \otimes \dots \otimes U = U^{\otimes n}$  <br>
$\otimes$ => kronecker power atau kpow(Ka-Pow) 


In [ ]:
# how to use kronecker power  as a  function

def kpow(self, n: int) -> Tensor :
    """Return the tensor product with itself `n` times."""

    if n == 0:
        return 1.0
    t = self
    for _ in range (n-1):
        t = np.kron(t,self)
    return self.__class__(t) # Necessary to return a Tensor type

### Perbandingan hasil tensor1 dan tensor2  
Masalah jika melakukan secara manual (==) :
 - Floating-point precision issues => komputer tidak dapat menyimpan sebagian besar angka desimal secara sempurna (ex. 0.1 + 0.2 = 0.300000004)
 - $\epsilon$ (epsilon) / Tolerance => tidak bisa membandingkan kesetaraan secara eksak, solusinya adalah memeriksa apakah selisih antara dua angka "cukup kecil" (ex. $10^{-6} = 1e^{-6}$)
<br><br>
Dalam numpy terdapat fungsi allclose() yang mempermudah kode untuk membandingkan tensor <br>
<i><b> all close <b><i> memeriksa apakah perbedaan antara elemen-elemen berada dalam <b>batas toleran<b> yang sangat kecil? dibanding harus memberikan perbandingan (==)

In [ ]:
def is_close(self, arg) -> bool:
    """Check that a 1D or 2D tensor is numerically close to arg"""

    return np.allclose(self,arg, atol=1e-6)

# memeriksa apakah tensor self cukup mirip dengan tensor arg dengan batas toleransi 1e-6

In [17]:
# Check if this tensor is Hermitian - Udag = U?
def is_Hermitians(self) -> bool :
    
    if len(self.shape) != 2:
        return False
    if self.shape[0] != self.shape[1]:
        return  False
    return self.isclose(np.conj(self.transpose()))

# check if this tensor is unitary - Udag*U = I?
def is_Unitary(self) -> bool :
    return Tensor(np.conj(self.transpose()) @ self).is_close(
        Tensor(np.eye(self.shape[0]))
    )

In [18]:
def is_permutation(self) -> bool:
    x = self
    return (x.ndim == 2 and x.shape[0] == x.shape[1] and
    (x.sum(axis=0) == 1).all() and
    (x.sum(axis=1) == 1).all() and
    ((x == 1) or (x == 0)).all())